In [1]:
import numpy as np
from math import inf

In [2]:
def find_identity(size, matrix):
    base = []
    pos = []
    for col in range(len(matrix[0])):
        zero_col = True
        one_col = False
        temp_pos = 0
        for line in range(len(matrix)):
            if line < len(matrix) - size and matrix[line][col] != 0:
                zero_col = False
                break
            if matrix[line][col] == 1:
                if one_col:
                    zero_col = False
                    break
                else:
                    if line not in pos: 
                        one_col = True
                        temp_pos = line
                    else:
                        zero_col = False
                        break
            else:
                if matrix[line][col] != 0:
                    zero_col = False
                    break
        if zero_col and one_col:
            base.append(col)
            pos.append(temp_pos)#Linha
    return([base, pos])#Coluna, Linha

In [3]:
def cria_base(size, m, I):
    #Acha semi-base
    new_matrix = m[1::]
    base = find_identity(size, new_matrix) #Col/line
    for i in range(len(base[1])):
        base[1][i] += 1
        
    #Limpa base
    for col in range(len(base[0])):
        for line in range(len(m)):
            if line != base[1][col]:
                if m[line][base[0][col]] != 0:
                    for l in range(len(m)):
                        if l != line and m[l][base[0][col]] != 0:
                            a = m[l][base[0][col]]/m[line][base[0][col]]
                            m[line] = list(np.array(m[l])*a - np.array(m[line]))
    
    run_base = True
    old_bloqueio = []
    while run_base:
        #Coluna de maior valor da função
        col = list(m[0]).index(min(m[0][0:-1]))
        #print(f"col = {col}")

        #Cria list com valores de linha bloqueio
        linha_bloqueio = []
        for i in range(2, len(m)):
            if m[i][col] > 0:
                linha_bloqueio.append(m[i][-1]/m[i][col])
            else:
                #print(f"m[i] = {m[i]}, m[i][col] = {m[i][col]}")
                linha_bloqueio.append(inf)
        #print(f"linha_bloqueio = {linha_bloqueio}")

        linha_pivo = (linha_bloqueio.index(min(linha_bloqueio)) + 2)
        #print(f"linha_pivo = {linha_pivo}")

        for line in range(len(m)):
            if line != linha_pivo:
                a = 0
                if m[linha_pivo][col] != 0:
                    a = -m[line][col]/m[linha_pivo][col]
                m[line] = list(np.array(m[linha_pivo])*a + np.array(m[line]))
                #print(f"line = {line}, a = {a},  m[line] = {m[line]}")
        
        a = 1/m[linha_pivo][col]
        m[linha_pivo] = list(np.array(m[linha_pivo])*a)
        #print(f"a m = {np.array(m)}\n\n")
        
        if old_bloqueio == linha_bloqueio:
            run_base = False
            break
        old_bloqueio = linha_bloqueio
    
    m = np.delete(np.array(m), 0, 0)
    m = np.delete(np.array(m), -2, 1)
    m = np.delete(np.array(m), -2, 1)
    
    new_matrix = []
    for i in m:
        temp = []
        for j in i:
            temp.append(j)
        new_matrix.append(temp)
    return new_matrix

In [4]:
def find_base(matrix, matrix_aux):
    matrix_return = matrix
    num_restricao = len(matrix)-1
    I = find_identity(num_restricao, matrix)
    if len(I[0]) != num_restricao:
        I = find_identity(num_restricao, matrix_aux)
        if len(I[0]) != num_restricao:
            if len(I[0]) == 0:
                print("Não foi possivel fazer o pivoteamento")
            else:
                matrix_return = cria_base(num_restricao, matrix_aux, I)
                I = find_identity(num_restricao, matrix_return)
                if len(I[0]) != num_restricao:
                    print("Não foi possivel achar a base")
    return list(matrix_return)

In [5]:
def simplex(matrix, matrix_aux):
    m = find_base(matrix, matrix_aux)
    #print(f"m = {np.array(m)}\n\n\n")
    run_simplex = True
    old_bloqueio = []
    while run_simplex:
        #Numero maximo de linhas
        max_line = max(m[0])
        
        #Coluna de maior valor da função
        col = list(m[0]).index(max_line)
        #print(f"col = {col}")
        
        #Cria list com valores de linha bloqueiov
        linha_bloqueio = []
        for i in range(1, len(m)):
            if m[i][col] > 0:
                linha_bloqueio.append(m[i][-1]/m[i][col])
            else:
                linha_bloqueio.append(inf)
        #print(f"linha_bloqueio = {linha_bloqueio}")
        
        linha_pivo = (linha_bloqueio.index(min(linha_bloqueio)) + 1)
        #print(f"linha_pivo = {linha_pivo}")
        
        for line in range(len(m)):
            if line != linha_pivo:
                a = 0
                if m[linha_pivo][col] != 0:
                    a = -m[line][col]/m[linha_pivo][col]
                m[line] = list(np.array(m[linha_pivo])*a + np.array(m[line]))
                #print(f"line = {line}, a = {a},  m[line] = {m[line]}")
                
        #print(f"m[linha_pivo][col] = {m[linha_pivo][col]}, m[linha_pivo] = {m[linha_pivo]}")
        a = 1/m[linha_pivo][col]
        m[linha_pivo] = list(np.array(m[linha_pivo])*a)
        #print(f"m = {np.array(m)} \n\n\n\n")
        
        if old_bloqueio == linha_bloqueio:
            run_simplex = False
                
        old_bloqueio = linha_bloqueio
    
    
    pos_x = []
    val_x = []
    for col in range(len(m[0])):
        if m[0][col] == 0:
            for line in range(1, len(m)):
                if m[line][col] == 1:
                    pos_x.append(col)
                    val_x.append(m[line][-1])
                    
    for i in range(len(m[0])-1):
        if i in pos_x:
            print(f"resultado otimizado -> x_{i + 1} = {val_x[pos_x.index(i)]}")
        else:
            print(f"resultado otimizado -> x_{i + 1} = 0")
    return [pos_x, val_x]

In [6]:
matrix = [[-1, 2, 0, 0, 0, 0],
          [ 1, 1,-1, 0, 0, 2],
          [-1, 1, 0,-1, 0, 1],
          [ 0, 1, 0, 0, 1, 3]]

matrix_aux = [[ 0, 0, 0, 0, 0, 1, 1, 0],
              [-1, 2, 0, 0, 0, 0, 0, 0],
              [ 1, 1,-1, 0, 0, 1, 0, 2],
              [-1, 1, 0,-1, 0, 0, 1, 1],
              [ 0, 1, 0, 0, 1, 0, 0, 3]]

# matrix = [[4, 3, 0, 0, 0, 0 ],
#           [2, 1, 1, 0, 0, 80],
#           [1, 2, 0, 1, 0, 70],
#           [0, 1, 0, 0, 1, 30]]

# matrix_aux = matrix

simplex(matrix, matrix_aux)

resultado otimizado -> x_1 = 0
resultado otimizado -> x_2 = 3.0
resultado otimizado -> x_3 = 1.0
resultado otimizado -> x_4 = 2.0
resultado otimizado -> x_5 = 0


[[1, 2, 3], [3.0, 1.0, 2.0]]